In [1]:
# To allow changes in .py files to be reflected in Notebook
%load_ext autoreload
%autoreload 2

Randomly select streams to use for drift stream generation. Check summary statistics to determine value for n_drift during drift generation. If trouble is encountered with generating large percentage drift, using a selection of streams with a smaller mean for num_anomalies (therefore requiring a smaller n_drift) may help.

In [1]:
from util.generate_moa_stream import GenMOAStream
import numpy as np
import pandas as pd

In [2]:
source_dir = '/Users/tammyz/Desktop/3H03/AnomalyDriftDetection/data/benchmark/ECG'
drift_dir = '/Users/tammyz/Desktop/3H03/AnomalyDriftDetection/data/synthetic'
moa_path = '/Users/tammyz/Desktop/3H03/moa-release-2023.04.0/lib'
num_streams = 6
g = GenMOAStream(source_dir, drift_dir, moa_path, num_streams)

In [3]:
df = g.get_source_summary()
df

,filename,len,num_anomalies,total_anom_cover,avg_anomaly_len,percent_anomalies
1,MBA_ECG14046_data_1.arff,229900,447,36478,81.606264,0.158669
4,MBA_ECG14046_data_12.arff,229900,169,14318,84.721893,0.062279
14,MBA_ECG14046_data_21.arff,229900,95,7229,76.094737,0.031444
16,MBA_ECG14046_data_23.arff,229900,124,9300,75.000000,0.040452
18,MBA_ECG14046_data_25.arff,229900,106,8241,77.745283,0.035846
22,MBA_ECG14046_data_29.arff,229900,63,4725,75.000000,0.020552


In [4]:
df.describe()

,len,num_anomalies,total_anom_cover,avg_anomaly_len,percent_anomalies
count,6.0,6.000000,6.000000,6.000000,6.000000
mean,229900.0,167.333333,13381.833333,78.361363,0.058207
std,0.0,141.402499,11749.016766,3.977446,0.051105
min,229900.0,63.000000,4725.000000,75.000000,0.020552
25%,229900.0,97.750000,7482.000000,75.273684,0.032545
50%,229900.0,115.000000,8770.500000,76.920010,0.038149
75%,229900.0,157.750000,13063.500000,80.641019,0.056823
max,229900.0,447.000000,36478.000000,84.721893,0.158669


In [5]:
mean_num_anom = 168
percent = np.arange(0.1,0.8,0.1)
percent_anom = mean_num_anom * percent
pd.DataFrame([percent_anom], columns=percent)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7
0,16.8,33.6,50.4,67.2,84.0,100.8,117.6


In [11]:
length = 229900
n_drift = 140
p_drift = 0.39
p_drift_before = 0.5
sub_dir = 'n_drift/n_a60'
dataset = 'ECG'

In [12]:
output_path, drift_label, streams, positions, seq_before, w_drift = g.run_generate_grad_stream_moa(
    length, 
    p_drift, 
    n_drift,
    p_drift_before, 
    sub_dir,
    dataset
)

Generating splits...
	Getting partitions...
	Getting order of drifts coming before anomaly...
	Getting drift center positions...
	Getting stream file cuts...
Done!
Getting stream file cuts...	Done!
Creating intermediate files...	

Done!
Recursively generating MOA command...	Done!
Drift filename:  ECG_grad_p38_n100_b50
Running terminal command...	


{M}assive {O}nline {A}nalysis
Version:  23.04 April 2023
Copyright: (C) 2007-2023 University of Waikato, Hamilton, New Zealand
Web: http://moa.cms.waikato.ac.nz/

                                                                               
Task completed in 2.21s (CPU time)



Stream written to ARFF file /Users/tammyz/Desktop/3H03/AnomalyDriftDetection/data/synthetic/n_drift/n_a60/ECG_grad_p38_n100_b50.arff
Done!
Generating drift labels...	Done!


Tweak Existing Dataset

In [511]:
from util.plot_stream import PlotStream
from util.create_drift import find_anomaly_intervals

source_dir = "data/benchmark/ECG" # directory for source streams
drift_dir = "data/synthetic/n_drift/n_a50" # directory for drift stream
filename = "ECG_grad_p37_n98_b50"
d1 = PlotStream(source_dir, drift_dir, filename)

In [512]:
g2 = GenMOAStream(source_dir, drift_dir, moa_path, selected_streams=d1.source_streams)
positions = d1.positions[1:]
streams = d1.streams
seq_before = d1.seq_before
w_drift = d1.w_drift

In [18]:
len(positions), len(streams), len(seq_before), len(w_drift)

(100, 100, 139, 139)

In [14]:
positions = [0] + positions

In [53]:
k = 4
w = 3
index = [i for i in range(k-w,k+w,1)]
pd.DataFrame({'positions': positions[k-w:k+w], 'streams (after)': streams[k-w:k+w], 'seq_before': seq_before[k-w-1:k+w-1], 'w_drift': w_drift[k-w-1:k+w-1]}, index=index)

,positions,streams (after),seq_before,w_drift
1,1714,0,True,71
2,4623,2,False,1201
3,5992,3,False,361
4,6591,0,True,639
5,8017,2,True,312
6,10024,4,True,508


In [38]:
streams[60]

4

In [51]:
positions[34] = 76500

In [46]:
w_drift[53] = 250

In [31]:
seq_before[84] = True

In [54]:
# r_i = 3
positions = positions[:r_i] + positions[r_i+1:]
streams = streams[:r_i] + streams[r_i+1:]
w_drift[r_i] = w_drift[r_i-1] + w_drift[r_i]
w_drift = w_drift[:r_i-1] + w_drift[r_i:]
seq_before = seq_before[:r_i-1] + seq_before[r_i:]

In [390]:
index = [i for i in range(k-w,k+w,1)]
pd.DataFrame({'positions': positions[k-w:k+w], 'streams (after)': streams[k-w:k+w], 'w_drift': w_drift[k-w-1:k+w-1]}, index=index)

,positions,streams (after),w_drift
30,98398,5,344
31,108581,4,5097
32,110528,0,24
33,116615,4,1675


In [55]:
output_path, drift_label =  g.construct_drift_stream(
    positions[1:],
    streams,
    w_drift,
    seq_before,
    sub_dir='n_drift/n_a60',
    length=229900,
    dataset='ECG'
)

Getting stream file cuts...	Done!
Creating intermediate files...	Done!
Recursively generating MOA command...	Done!
Drift filename:  ECG_grad_p37_n96_b52
Running terminal command...	


{M}assive {O}nline {A}nalysis
Version:  23.04 April 2023
Copyright: (C) 2007-2023 University of Waikato, Hamilton, New Zealand
Web: http://moa.cms.waikato.ac.nz/

                                                                               
Task completed in 1.39s (CPU time)



Stream written to ARFF file /Users/tammyz/Desktop/3H03/AnomalyDriftDetection/data/synthetic/n_drift/n_a60/ECG_grad_p37_n96_b52.arff
Done!
Generating drift labels...	Done!
